In [1]:
!pip install importlib
!pip install pdfminer3k

  Created wheel for importlib: filename=importlib-1.0.4-cp36-none-any.whl size=5858 sha256=ae0d4a17d773d011ece406e9c502643f0125d6acdfb88791fa6fbd2eff1e2d8a
  Stored in directory: /root/.cache/pip/wheels/26/eb/96/a8a055637794153edb35b7f7f97de528350367d42d8829c187
Successfully built importlib
     |████████████████████████████████| 4.1MB 4.8MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
  Created wheel for pdfminer3k: filename=pdfminer3k-1.3.1-cp36-none-any.whl size=105356 sha256=5d4d1dc0277cf14f52c752c67ffafb63352101e50ac1b9e726643b9831f997f1
  Stored in directory: /root/.cache/pip/wheels/ca/4f/a7/cb601b4fb257d2321ac668b7c6e269176780bd0283eda855d2
Successfully built pdfminer3k


In [4]:

import sys
import importlib
importlib.reload(sys)
import re

from pdfminer.pdfparser import PDFParser,PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTTextBoxHorizontal,LAParams
from pdfminer.pdfinterp import PDFTextExtractionNotAllowed
#from pdfminer.pdfpage import PDFPage
from pdfminer.pdfdevice import PDFDevice

from urllib.request import urlopen

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/'My Drive'/'Peng' 


!ls "/content/drive/My Drive/Peng"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Peng
code.docx						  PdfToText-New.ipynb
DiagnosticandstatisticalmanualofmentaldisordersDSM-5.pdf  Untitled
instruction1.pdf					  Untitled0.ipynb
instruction2.pdf


In [5]:

pdf_path = 'DiagnosticandstatisticalmanualofmentaldisordersDSM-5.pdf'
newname1 = 'newtest.txt'

def extract_layout_by_page(pdf_path):
   
    laparams = LAParams() # pdfminer.layout.LAParams(line_overlap=0.5, char_margin=2.0, line_margin=0.5, word_margin=0.1, boxes_flow=0.5, detect_vertical=False, all_texts=False)

    fp = open(pdf_path, 'rb')  # 
    parser = PDFParser(fp)   # Create a PDF parser object associated with the file object. 
    document = PDFDocument(parser) # Create a PDF document object that stores the document structure.
    
    
    #Connect analyzer to document object
    parser.set_document(document)
    document.set_parser(parser)   
    
    # Check if the document allows text extraction. If not, abort 
    document.initialize()
    if not document.is_extractable:
        raise PDFTextExtractionNotAllowed
   
    rsrcmgr = PDFResourceManager()  # Create a PDF resource manager object that stores shared resources
   
    #device = PDFDevice(rsrcmgr) #'PDFDevice' object has no attribute 'get_result'
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)  # Create a PDF page aggregator object
    interpreter = PDFPageInterpreter(rsrcmgr, device) # Create a PDF interpreter object

    layouts = []
    count = 0
    
    # Process each page contained in the document 
    for page in document.get_pages():
        if count < 100:
            interpreter.process_page(page)
            layouts.append(device.get_result()) 
            count = count + 1
        else:
            break
    return layouts

def pdf_to_txt(pdf_path, newname1):
    page_layouts = extract_layout_by_page(pdf_path)
    with open(newname1, 'w', encoding='utf-8') as f:
        for current_page in page_layouts:
            # Get text
            for x in current_page:
                if hasattr(x, "get_text"):
                    # result.append(x.get_text())
                    results = x.get_text()
                    # print(type(results))
                    if results == ' \n':
                        continue
                    # results=list(results)
                    # with open(newname1, 'a', encoding='utf-8') as f:
                    f.write(results)
                    # f.close()

    
    with open(newname1, 'r', encoding='UTF-8-sig') as f:
        content = f.readlines()
        print(content)
    pattern = re.compile(r'\s{1,2}\n')
    newcontent = []
    for i in content:
        if bool(re.search(pattern, i)) is False:
            j = i.replace('\n', '').replace(' \n', '')
            newcontent.append(j)
        else:
            newcontent.append(i)
    f.close()
    print(newcontent)
    with open(newname1, 'w', encoding='UTF-8') as f:
        for i in newcontent:
            f.write(i)
    f.close()
    return newname1

pdf_to_txt(pdf_path, newname1)

PSEOF: ignored